In [17]:
# Setup

from requests import get, post, put, delete
from blowfish import Cipher
from ast import literal_eval

url = "http://192.168.0.10:5000"

def get_cipher_key(token: str) -> bytes:
    data = {
        "token": token
    }

    key = get(url + "/key", json=data).text
    return str.encode(key)


def get_iv(token: str) -> bytes:
    data = {
        "token": token
    }

    iv = get(url + "/iv", json=data).text
    return str.encode(iv)


def encrypt_title(cipher: Cipher, title: str, iv: str) -> str:
    encrypted_title = b"".join(cipher.encrypt_cfb(title, iv))

    return str(encrypted_title)

def decrypt_title(cipher: Cipher, encrypted_title: str, iv: str) -> str:
    encrypted_title = literal_eval(encrypted_title)
    decrypted_title = b"".join(cipher.decrypt_cfb(encrypted_title, iv))

    return decrypted_title.decode()

In [30]:
# Demonstration - Subnote Creation

def main():
    token = input("Token: ").strip().lower()

    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return

    key = get_cipher_key(token)
    iv = get_iv(token)
    cipher = Cipher(key)

    parent_note_id = int(input("Parent Note ID: "))
    content = str.encode(input("Content: ").strip())

    data = {
        "token": token,
        "note_id": parent_note_id,
        "content": encrypt_title(cipher, content, iv)

    }
        
    response = post(url + "/subnotes", json=data)
    print(response.text)

main()

Token:  4b5de8fdda7fc9b8ddaf387b0c56415fbac636abaa4af2b4c41b912b4400267c
Parent Note ID:  1
Content:  I was feeling indifferent and then I felt sad. Suddenly, I was excited for the day and felt good about myself. 


POST /subnotes


In [32]:
# Demonstration - Updating a Subnote

def main():
    token = input("Token: ").strip().lower()

    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return

    key = get_cipher_key(token)
    iv = get_iv(token)
    cipher = Cipher(key)

    note_id = int(input("Note ID: "))
    subnote_id = int(input("Subnote ID: "))
    content = str.encode(input("Updated Content: ").strip())

    data = {
        "token": token,
        "note_id": note_id,
        "subnote_id": subnote_id,
        "content": encrypt_title(cipher, content, iv)
    }

    response = put(url + "/subnotes", json=data)
    print(response.text)

main()

Token:  4b5de8fdda7fc9b8ddaf387b0c56415fbac636abaa4af2b4c41b912b4400267c
Note ID:  1
Subnote ID:  2
Updated Content:  I was feeling indifferent and then I felt bad. Suddenly, I was excited for the day and felt good about myself.


Subnote updated.


In [28]:
# Demonstration - Subnote Deletion

def main():
    token = input("Token: ").strip().lower()

    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return

    parent_note_id = int(input("Parent Note ID: "))
    subnote_id = int(input("Subnote ID: "))

    data = {
        "token": token,
        "note_id": parent_note_id,
        "subnote_id": subnote_id
    }

    response = delete(url + "/subnotes", json=data)
    print(response.text)

main()

Token:  4b5de8fdda7fc9b8ddaf387b0c56415fbac636abaa4af2b4c41b912b4400267c
Parent Note ID:  1
Subnote ID:  2


Subnote deleted.


In [31]:
# Demonstration - Retrieve Subnotes

def main():
    token = input("Token: ").strip().lower()

    data = {
        "token": token
    }

    response = post(url + "/validate-token", json=data)

    if response.text == "0":
        print("Invalid token.")
        return

    key = get_cipher_key(token)
    iv = get_iv(token)
    cipher = Cipher(key)
    
    response = get(url + "/subnotes", json=data)

    for subnote in response.json():
        subnote_id = subnote["id"]
        content = decrypt_title(cipher, subnote["content"], iv)
        modified = subnote["modified"]
        parent_note_id = subnote["note_id"]

        print("")
        
        print(f"Subnote ID: {subnote_id}")
        print(f"Content: {content}")
        print(f"Modified: {modified}")
        print(f"Parent Note ID: {parent_note_id}")

main()

Token:  4b5de8fdda7fc9b8ddaf387b0c56415fbac636abaa4af2b4c41b912b4400267c



Subnote ID: 1
Content: The third studio album by The Prodigy.
Modified: 1709222379
Parent Note ID: 1

Subnote ID: 2
Content: I was feeling indifferent and then I felt sad. Suddenly, I was excited for the day and felt good about myself.
Modified: 1709238517
Parent Note ID: 1
